In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from itertools import product
from functools import partial
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('nb-raw_debug-raw-2.ipynb', 'raw' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize, float64, uint
# from tslearn.preprocessing import TimeSeriesScalerMeanVariance
# from tslearn.piecewise import PiecewiseAggregateApproximation
# from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from scipy.stats import zscore
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 200)
pd.set_option('display.max_columns', 50)

from common_util import RAW_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_FMT_YMD_HM, DT_BIZ_DAILY_FREQ, series_to_dti_noreindex, df_rows_in_year, timestamp_on, dt_now, get_custom_biz_freq, get_custom_biz_freq_df, query_df, search_df, chained_filter, benchmark
from common_util import MUTATE_DIR, load_df, load_json, outer_join, left_join, count_nn_df, count_nz_df, count_nn_nz_df, pairwise, cust_count, list_get_dict, get_time_mask
from data.data_api import DataAPI
from raw.common import default_row_masksfile
from recon.viz import *
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/raw/nb-raw_debug-raw-2.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load Data

### RAW

In [2]:
VOL_DIR = RAW_DIR +'vol/'
PRICE_DIR = RAW_DIR +'price/'

SPX = load_df(PRICE_DIR +'SPX.parquet')
RUT = load_df(PRICE_DIR +'RUT.parquet')
NDX = load_df(PRICE_DIR +'NDX.parquet')
DJI = load_df(PRICE_DIR +'DJI.parquet')
VIX = load_df(VOL_DIR +'VIX.parquet')
RVX = load_df(VOL_DIR +'RVX.parquet')
VXN = load_df(VOL_DIR +'VXN.parquet')
DVX = load_df(VOL_DIR +'VXD.parquet')

In [3]:
pba = {
    'sp_500': SPX,
    'russell_2000': RUT,
    'dow_jones': DJI,
    'nasdaq_100': NDX,    
}
vol = {
    'sp_500': VIX,
    'russell_2000': RVX,
    'dow_jones': DVX,
    'nasdaq_100': VXN,    
}

In [4]:
def get_local_dti(dti, dest_tz='US/Eastern'):
    conv_dti = dti.tz_convert(dest_tz)
    return conv_dti

def filter_day(day_df, cutoff_time=timestamp_on(dt_now())(hour=8), offset_unit=DT_HOURLY_FREQ):
    """
    Filter function to remove days that don't start at or before the passed cutoff time.

    Args:
        day_df (pd.DataFrame): intraday dataframe
        cutoff_time (pd.Timestamp): pandas timestamp, only the time components are used
        offset_col (str): offset column to convert the UTC index to local time
        offset_unit (str): offset unit of the offset column
    
    Returns:
        None or pd.DataFrame
    """
    unique_dates = list(set(day_df.index.date))
    assert(len(unique_dates)==1)
    date = unique_dates[0]
    local_times = get_local_dti(day_df.dropna(how='all').index)
    valid_local_times = local_times[local_times.date==date]

    if (not all(pd.isnull(local_times))):
        min_time = local_times.min()
        cutoff_time = timestamp_on(min_time)(hour=cutoff_time.hour, minute=cutoff_time.minute, second=cutoff_time.second)
        if (min_time <= cutoff_time):
            return day_df
    return None

def filt(day_df, cutoff_time=timestamp_on(dt_now())(hour=9), offset_unit=DT_HOURLY_FREQ):
    """
    Filter function to remove days that don't start at or before the passed cutoff time.

    Args:
        day_df (pd.DataFrame): intraday dataframe
        cutoff_time (pd.Timestamp): pandas timestamp, only the time components are used
        offset_col (str): offset column to convert the UTC index to local time
        offset_unit (str): offset unit of the offset column
    
    Returns:
        None or pd.DataFrame
    """
    unique_dates = list(set(day_df.index.date))
    assert(len(unique_dates)==1)
    date = unique_dates[0]
    local_times = get_local_dti(day_df.dropna(how='all').index)
    valid_local_times = local_times[local_times.date==date]

    if (not all(pd.isnull(valid_local_times))):
        min_time = valid_local_times.min()
        cutoff_time = timestamp_on(min_time)(hour=cutoff_time.hour, minute=cutoff_time.minute, second=cutoff_time.second)
        return min_time <= cutoff_time
    return False

In [5]:
from common_util import pd_before_cutoff

In [6]:
ass = 'russell_2000'
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM
joined = pba[ass]

joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
joined = joined.asfreq(dti_freq)

In [7]:
joined.groupby(pd.Grouper(freq=agg_freq)).filter(pd_before_cutoff)

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
1996-01-02 14:00:00+00:00,-6.0,315.9700,315.9700,314.7200,314.7500,315.2594,122.0
1996-01-02 15:00:00+00:00,-6.0,314.7200,315.2900,314.6800,315.2800,315.0755,239.0
1996-01-02 16:00:00+00:00,-6.0,315.3100,315.4900,314.8400,314.8900,315.1048,240.0
1996-01-02 17:00:00+00:00,-6.0,314.8900,314.9600,314.7600,314.8700,314.8732,241.0
1996-01-02 18:00:00+00:00,-6.0,314.8600,315.2200,314.8300,315.1400,315.1353,240.0
...,...,...,...,...,...,...,...
2018-01-08 17:00:00+00:00,-5.0,1555.3049,1558.2067,1555.3049,1558.2067,1556.8731,720.0
2018-01-08 18:00:00+00:00,-5.0,1558.2186,1561.7482,1558.2186,1561.6385,1560.7032,720.0
2018-01-08 19:00:00+00:00,-5.0,1561.6467,1562.8080,1561.6467,1562.7438,1562.4006,720.0


In [8]:
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM

for ass in ['sp_500', 'russell_2000', 'dow_jones', 'nasdaq_100']:
    for i, raw_dti_df in enumerate([pba[ass], vol[ass]]):
        joined = raw_dti_df
        joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
        joined = joined.asfreq(dti_freq)
        
        res = joined.groupby(pd.Grouper(freq=agg_freq)).filter(pd_before_cutoff)
        a = df_rows_in_year(res).dropna(how='all')
        b = df_rows_in_year(raw_dti_df).dropna(how='all')
        d = b.index.difference(a.index).date
        display(ass, str(i))
        display(get_local_dti(a.index).hour.value_counts())
        display(get_local_dti(b.index).hour.value_counts())
        display(pd.Series(d).value_counts())

/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:456: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()
/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:464: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  dindex = axis_index.to_series()


'sp_500'

'0'

13    2265
12    2265
11    2265
10    2265
9     2265
14    2254
16    2248
15    2247
17      65
18       3
4        1
Name: id, dtype: int64

13    2265
12    2265
11    2265
10    2265
9     2265
14    2254
16    2248
15    2247
17      65
18       3
4        1
Name: id, dtype: int64

...

'nasdaq_100'

'1'

14    2258
13    2258
11    2258
9     2258
12    2257
10    2257
16    2241
15    2240
18       3
17       2
4        1
Name: id, dtype: int64

14    2259
13    2259
11    2259
12    2258
10    2258
9     2258
16    2242
15    2241
18       3
17       2
4        1
Name: id, dtype: int64

2014-11-19    7
dtype: int64

In [110]:
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM

for ass in ['sp_500', 'russell_2000', 'dow_jones', 'nasdaq_100']:
    for i, raw_dti_df in enumerate([pba[ass], vol[ass]]):
        joined = raw_dti_df
        joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
        joined = joined.asfreq(dti_freq)
        
        res = joined.groupby(pd.Grouper(freq=agg_freq)).filter(filt)
        #res.index = res.index.droplevel(level=0)
        a = df_rows_in_year(res).dropna(how='all')
        b = df_rows_in_year(raw_dti_df).dropna(how='all')
        d = b.index.difference(a.index).date
        display(ass, str(i))
        display(get_local_dti(a.index).hour.value_counts())
        display(get_local_dti(b.index).hour.value_counts())
        display(pd.Series(d).value_counts())

'sp_500'

'0'

13    2265
12    2265
11    2265
10    2265
9     2265
14    2254
16    2248
15    2247
17      65
18       3
4        1
Name: id, dtype: int64

13    2265
12    2265
11    2265
10    2265
9     2265
14    2254
16    2248
15    2247
17      65
18       3
4        1
Name: id, dtype: int64

Series([], dtype: int64)

...

'nasdaq_100'

'1'

14    2258
13    2258
11    2258
9     2258
12    2257
10    2257
16    2241
15    2240
18       3
17       2
4        1
Name: id, dtype: int64

14    2259
13    2259
11    2259
12    2258
10    2258
9     2258
16    2242
15    2241
18       3
17       2
4        1
Name: id, dtype: int64

2014-11-19    7
dtype: int64

In [103]:
ass = 'russell_2000'
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM
joined = pba[ass]

joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
joined = joined.asfreq(dti_freq)

day_df = joined['2015-09-02']
unique_dates = list(set(day_df.index.date))
assert(len(unique_dates)==1)
date = unique_dates[0]

local_times = get_local_dti(day_df.dropna(how='all').index)
valid = local_times[local_times.date==date]
#local_times = list(filter(lambda dt: dt.date==date, get_local_dti(day_df.dropna(how='all').index)))
#local_times = [dt for dt in get_local_dti(day_df.dropna(how='all').index) if (dt.date==unique_dates[0])]

In [104]:
valid

DatetimeIndex(['2015-09-02 10:00:00-04:00', '2015-09-02 11:00:00-04:00',
               '2015-09-02 12:00:00-04:00', '2015-09-02 13:00:00-04:00',
               '2015-09-02 14:00:00-04:00', '2015-09-02 15:00:00-04:00',
               '2015-09-02 16:00:00-04:00', '2015-09-02 18:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', name='id', freq=None)

In [94]:
get_local_dti(joined['2010-10-05'].dropna().index)[get_local_dti(joined['2010-10-05'].dropna().index).date==date]

DatetimeIndex(['2010-10-05 09:00:00-04:00', '2010-10-05 10:00:00-04:00',
               '2010-10-05 11:00:00-04:00', '2010-10-05 12:00:00-04:00',
               '2010-10-05 13:00:00-04:00', '2010-10-05 14:00:00-04:00',
               '2010-10-05 15:00:00-04:00', '2010-10-05 16:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', name='id', freq=None)

### ROOT

In [4]:
root_rcs, root_dfs = DataAPI.axe_load(['root', 'root'])
root_split_rcs, root_split_dfs = DataAPI.axe_load(['root', 'root_split_ohlca'])
hrm_rcs, hrm_dfs = DataAPI.axe_load(['hrm', 'hrm'], lazy=False)

In [55]:
dta = root_dfs[[ass, 'root', 'root', 'join']].index.dt.year > 2009 or root_dfs[[ass, 'root', 'root', 'join']].index.dt.year < 2019
root_dfs[[ass, 'root', 'root', 'join']].dt.year

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,etf2_ver,etf2_buzz_N,etf2_relativeBuzz_N,etf2_sentiment_N,etf2_optimism_N,etf2_fear_N,etf2_joy_N,etf2_trust_N,etf2_violence_N,etf2_conflict_N,etf2_gloom_N,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,20221.2,0.049862,-0.167893,-0.013006,0.004747,0.006800,-0.000531,0.025790,0.013476,0.048316,...,0.061070,0.006009,0.010287,0.025873,0.017677,0.525497,-0.007905,0.006181,0.001381,0.001508,0.001172,0.021899,0.000082,0.007301,-0.000007,0.001299,0.001097,0.004539,0.000090,0.002635,0.006502,0.002837,-0.000007,0.004591,0.000112
2009-01-01 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,19935.7,0.049511,-0.169043,-0.012039,0.004715,0.006747,-0.000639,0.022999,0.012816,0.047227,...,0.061011,0.005849,0.010131,0.026390,0.017254,0.526963,-0.007323,0.006603,0.001379,0.001470,0.001146,0.021603,0.000038,0.007176,-0.000023,0.001334,0.001070,0.004357,0.000090,0.002646,0.006219,0.002774,-0.000060,0.004726,0.000121
2009-01-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,19037.7,0.048142,-0.169243,-0.008903,0.005043,0.006697,-0.000946,0.024241,0.012922,0.046355,...,0.061434,0.005736,0.010450,0.026258,0.017788,0.529019,-0.006668,0.006506,0.001171,0.001281,0.001092,0.021954,0.000008,0.007291,-0.000024,0.001399,0.001312,0.004062,0.000094,0.002742,0.006026,0.002750,-0.000306,0.004840,0.000126
2009-01-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,18487.3,0.047380,-0.165411,-0.008141,0.004571,0.006897,-0.000839,0.025179,0.013252,0.045626,...,0.057153,0.005865,0.010423,0.026580,0.018673,0.518910,-0.006526,0.006543,0.001070,0.001323,0.001005,0.022482,-0.000025,0.007425,-0.000025,0.001503,0.001331,0.004337,0.000131,0.002834,0.006175,0.002875,-0.000065,0.004974,0.000131
2009-01-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,18250.2,0.047433,-0.168464,-0.009890,0.004301,0.006685,-0.000891,0.026328,0.013178,0.047205,...,0.057508,0.005856,0.010215,0.026587,0.018598,0.519227,-0.006582,0.006442,0.000895,0.001414,0.000970,0.022707,0.000067,0.007588,-0.000025,0.001539,0.001171,0.004233,0.000134,0.002870,0.006408,0.003028,-0.000167,0.005095,0.000134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.23,5322.5,0.035715,-0.083607,-0.001409,0.002912,0.006952,-0.001339,0.088304,0.028088,0.041428,...,0.049332,0.008996,0.025188,0.016084,0.031772,0.480336,0.007545,0.001331,0.001439,0.008780,0.001008,0.030110,0.001439,0.004354,0.000144,0.001583,0.004822,0.008852,NaN,0.000216,0.003094,0.002303,-0.001331,0.007053,0.000072
2017-12-31 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.23,5485.9,0.036388,-0.079932,-0.000273,0.002825,0.006015,-0.001413,0.091872,0.027252,0.042564,...,0.048894,0.009201,0.024609,0.016527,0.031754,0.481399,0.007646,0.000577,0.001407,0.008841,0.001010,0.030094,0.001516,0.004294,0.000144,0.001660,0.004547,0.010789,NaN,0.000217,0.002454,0.002309,-0.001407,0.007217,0.000072
2017-12-31 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.23,5525.9,0.035683,-0.078811,-0.

In [6]:
ass = 'russell_2000'
df_rows_in_year(root_dfs[[ass, 'root', 'root', 'join']])

TypeError: Cannot compare tz-naive and tz-aware datetime-like objects

## VIEW

In [24]:
ass = 'russell_2000'
root_df = root_dfs[[ass, 'root', 'root', 'join']]
ser = get_local_dti(root_df.iloc[:, 0].dropna().index).hour
ser.value_counts()

10    5484
11    5476
9     5476
12    5475
13    5473
14    5422
15    5414
16    5403
18       6
17       4
19       1
Name: id, dtype: int64

In [162]:
ass = 'russell_2000'
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM
joined = pba[ass]

joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
joined = joined.asfreq(dti_freq)

In [163]:
res = joined.groupby(pd.Grouper(freq=agg_freq)).apply(filter_day)
res.index = res.index.droplevel(level=0)

In [169]:
res

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
1996-01-02 14:00:00+00:00,-6.0,315.9700,315.9700,314.7200,314.7500,315.2594,122.0
1996-01-02 15:00:00+00:00,-6.0,314.7200,315.2900,314.6800,315.2800,315.0755,239.0
1996-01-02 16:00:00+00:00,-6.0,315.3100,315.4900,314.8400,314.8900,315.1048,240.0
1996-01-02 17:00:00+00:00,-6.0,314.8900,314.9600,314.7600,314.8700,314.8732,241.0
1996-01-02 18:00:00+00:00,-6.0,314.8600,315.2200,314.8300,315.1400,315.1353,240.0
...,...,...,...,...,...,...,...
2018-01-08 17:00:00+00:00,-5.0,1555.3049,1558.2067,1555.3049,1558.2067,1556.8731,720.0
2018-01-08 18:00:00+00:00,-5.0,1558.2186,1561.7482,1558.2186,1561.6385,1560.7032,720.0
2018-01-08 19:00:00+00:00,-5.0,1561.6467,1562.8080,1561.6467,1562.7438,1562.4006,720.0


In [148]:
df_rows_in_year(res).groupby(pd.Grouper(freq=agg_freq)).first().dropna(how='all')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
2009-01-02 00:00:00+00:00,-6.0,499.5100,500.6400,495.2700,495.5600,498.9440,120.0
2009-01-05 00:00:00+00:00,-6.0,505.5600,506.0300,496.0100,496.4600,500.0945,119.0
2009-01-06 00:00:00+00:00,-6.0,507.1600,513.6000,507.1600,513.5100,511.3483,120.0
2009-01-07 00:00:00+00:00,-6.0,510.3200,510.3200,502.7300,504.5600,504.7295,120.0
2009-01-08 00:00:00+00:00,-6.0,496.2500,496.2500,492.5200,493.6200,493.9059,120.0
...,...,...,...,...,...,...,...
2017-12-22 00:00:00+00:00,-5.0,1547.3676,1548.2539,1542.8533,1544.1095,1544.1546,358.0
2017-12-26 00:00:00+00:00,-5.0,1542.4220,1544.6425,1541.2135,1544.4663,1543.6791,358.0
2017-12-27 00:00:00+00:00,-5.0,1544.5409,1547.7295,1543.8019,1547.7295,1546.1849,358.0


In [79]:
day_df = joined.dropna(how='all').loc['2009-01-02']
local_times = get_local_dti(day_df.index)
if (not all(pd.isnull(local_times))):
    min_time = local_times.min()
    #cutoff_time = timestamp_on(min_time)(hour=cutoff_time.hour, minute=cutoff_time.minute, second=cutoff_time.second)
    #if (min_time <= cutoff_time):
    #        return day_df
    #return None

In [88]:
day_df.index.hour.value_counts()

15    1
14    1
21    1
20    1
19    1
18    1
17    1
16    1
Name: id, dtype: int64

In [78]:
day_df.index.min()

Timestamp('2009-01-02 14:00:00+0000', tz='UTC')

In [81]:
local_times.min()

Timestamp('2009-01-02 09:00:00-0500', tz='US/Eastern')

In [90]:
res = joined.groupby(pd.Grouper(freq=agg_freq)).apply(filter_day)
res.index = res.index.droplevel(level=0)

In [195]:
res.dropna(how='all').index.hour.value_counts()

14    5065
16    5059
15    5059
17    5054
18    5050
19    5017
20    4991
13    3100
21    2745
22     787
23     107
12       2
4        1
Name: id, dtype: int64

In [197]:
df_rows_in_year(res).dropna(how='all').index.hour.value_counts()

15    1848
14    1848
17    1848
16    1848
18    1847
19    1842
20    1831
13    1224
21     835
22     375
23     106
4        1
Name: id, dtype: int64

In [203]:
sorted(list(set(df_rows_in_year(res).dropna(how='all').index.hour)))

[4, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [193]:
df_rows_in_year(res.groupby(pd.Grouper(freq=agg_freq)).filter(lambda d: 21 in d.index.hour))

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
2009-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-02 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-02 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-02 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-02 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2017-12-29 19:00:00+00:00,-5.0,1545.9730,1545.9730,1542.2753,1542.2753,1544.005,720.0
2017-12-29 20:00:00+00:00,-5.0,1542.2765,1543.1239,1536.0876,1536.0952,1541.271,720.0
2017-12-29 21:00:00+00:00,-5.0,1535.9908,1535.9908,1535.5052,1535.5112,1535.537,121.0


In [190]:
res

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
1996-01-02 14:00:00+00:00,-6.0,315.9700,315.9700,314.7200,314.7500,315.2594,122.0
1996-01-02 15:00:00+00:00,-6.0,314.7200,315.2900,314.6800,315.2800,315.0755,239.0
1996-01-02 16:00:00+00:00,-6.0,315.3100,315.4900,314.8400,314.8900,315.1048,240.0
1996-01-02 17:00:00+00:00,-6.0,314.8900,314.9600,314.7600,314.8700,314.8732,241.0
1996-01-02 18:00:00+00:00,-6.0,314.8600,315.2200,314.8300,315.1400,315.1353,240.0
...,...,...,...,...,...,...,...
2018-01-08 17:00:00+00:00,-5.0,1555.3049,1558.2067,1555.3049,1558.2067,1556.8731,720.0
2018-01-08 18:00:00+00:00,-5.0,1558.2186,1561.7482,1558.2186,1561.6385,1560.7032,720.0
2018-01-08 19:00:00+00:00,-5.0,1561.6467,1562.8080,1561.6467,1562.7438,1562.4006,720.0


In [185]:
df_rows_in_year(res[res.index.hour==21])

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_avgPrice,pba_numTrades
id,,,,,,,
2009-01-02 21:00:00+00:00,-6.0,504.5800,505.8400,504.5800,505.8400,505.7680,240.0
2009-01-05 21:00:00+00:00,-6.0,504.5700,505.0300,504.5700,505.0300,505.0160,240.0
2009-01-06 21:00:00+00:00,-6.0,514.3200,514.7100,514.3200,514.7100,514.6907,240.0
2009-01-07 21:00:00+00:00,-6.0,497.2300,497.2300,497.1000,497.1000,497.1068,240.0
2009-01-08 21:00:00+00:00,-6.0,501.9500,502.0100,501.9500,502.0100,502.0061,240.0
...,...,...,...,...,...,...,...
2017-12-22 21:00:00+00:00,-5.0,1543.5459,1543.6464,1542.9254,1542.9254,1542.9779,121.0
2017-12-26 21:00:00+00:00,-5.0,1544.6516,1544.6829,1544.2223,1544.2264,1544.2699,121.0
2017-12-27 21:00:00+00:00,-5.0,1544.2396,1544.2396,1543.9347,1543.9369,1543.9507,121.0


In [176]:
res.groupby(pd.Grouper(freq=agg_freq)).agg(lambda d: d.iloc[-1].index.hour)

AttributeError: 'numpy.float64' object has no attribute 'index'

In [96]:
res.dropna(how='all').index.hour.value_counts()

14    5490
16    5484
15    5484
17    5479
18    5473
19    5437
20    5416
13    3374
21    2644
22     418
12       3
23       1
9        1
Name: id, dtype: int64

In [ ]:
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM
ass = 'russell_2000'

In [156]:
agg_freq = DT_CAL_DAILY_FREQ
dti_freq = DT_HOURLY_FREQ
dt_fmt = DT_FMT_YMD_HM

for ass in ['sp_500', 'russell_2000', 'dow_jones', 'nasdaq_100']:
    for i, raw_dti_df in enumerate([pba[ass]]):#, vol[ass]]):
        joined = df_rows_in_year(raw_dti_df)
        joined.index = series_to_dti_noreindex(joined.index, fmt=dt_fmt, utc=True, exact=True, freq=dti_freq)
        joined = joined.asfreq(dti_freq)
        
        res = joined.groupby(pd.Grouper(freq=agg_freq)).apply(filter_day)
        res.index = res.index.droplevel(level=0)
        display(ass, str(i))
        display(res.dropna(how='all').index.hour.value_counts())
        display(df_rows_in_year(raw_dti_df).dropna(how='all').index.hour.value_counts())

'sp_500'

'0'

15    2265
14    2265
17    2265
16    2265
18    2263
19    2252
20    2247
13    1492
21     802
22      26
9        1
Name: id, dtype: int64

15    2265
14    2265
17    2265
16    2265
18    2263
19    2252
20    2247
13    1492
21     802
22      26
9        1
Name: id, dtype: int64

'russell_2000'

...

15    2265
14    2265
17    2265
16    2265
18    2264
19    2250
20    2247
13    1720
21     788
12     486
22       6
0        1
Name: id, dtype: int64

'nasdaq_100'

'0'

15    2261
14    2261
17    2261
16    2261
18    2260
19    2257
20    2242
21    2149
13    1489
22     710
Name: id, dtype: int64

17    2264
16    2264
18    2263
15    2262
14    2261
19    2260
20    2245
21    2152
13    1489
22     711
Name: id, dtype: int64

## Hour Dist. in get_price.py data

In [43]:
dti_freq = DT_HOURLY_FREQ

for ass in ['sp_500', 'russell_2000', 'dow_jones', 'nasdaq_100']:
    for raw_dti_df in [pba[ass], vol[ass]]:
        raw_dti_df = df_rows_in_year(raw_dti_df)
        raw_dti_df.index = series_to_dti_noreindex(raw_dti_df.index, fmt=DT_FMT_YMD_HM, utc=True, exact=True, freq=dti_freq)
        raw_dti_df = raw_dti_df.asfreq(dti_freq)
        ser = get_local_dti(raw_dti_df.iloc[:, 0].dropna().index).hour
        display(ass)
        display(ser.value_counts().iloc[:9])

'sp_500'

13    2265
12    2265
11    2265
10    2265
9     2265
14    2254
16    2248
15    2247
17      65
Name: id, dtype: int64

'sp_500'

9     2265
10    2264
13    2262
12    2262
11    2261
14    2249
15    2245
16    2243
6      430
Name: id, dtype: int64

'russell_2000'

...

9     2264
10    2263
11    2236
13    2235
14    2231
12    2225
15    2218
16    2211
18       2
Name: id, dtype: int64

'nasdaq_100'

13    2264
12    2264
14    2263
11    2262
10    2262
9     2261
15    2246
16    2245
17    2103
Name: id, dtype: int64

'nasdaq_100'

10    2258
9     2258
12    2257
11    2257
14    2256
13    2256
15    2237
16    2235
18       2
Name: id, dtype: int64

In [28]:
ser = get_local_dti(raw_dti_df.iloc[:, 0].dropna().index).hour
ser.value_counts()

13    5495
12    5495
11    5492
10    5490
9     5482
14    5469
15    5436
16    5426
17    1367
22     512
18     368
19       8
20       5
21       4
8        3
23       1
0        1
Name: id, dtype: int64

In [ ]:
ex = raw_dti_df.loc[dayl[0]:dayl[1]].iloc[:, 1].dropna()
lt = get_local_dti(ex.index)

In [ ]:
filter_day(ex)

In [ ]:
raw_df.loc[dayl[0]:dayl[1]]

In [ ]:
ex

In [91]:
lt.min()

Timestamp('2009-01-02 09:00:00-0500', tz='US/Eastern')

In [64]:
agg_freq=DT_CAL_DAILY_FREQ
#cutoff_time = timestamp_on(dt_now())(hour=9)
clipped = raw_df#df_rows_in_year(out)
out=clipped.groupby(pd.Grouper(freq=agg_freq)).apply(partial(filter_day_old))
out.index = out.index.droplevel(level=0)

In [69]:
out.loc[dayl[0]:dayl[1]].dropna(how='all')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,etf2_ver,etf2_buzz_N,etf2_relativeBuzz_N,etf2_sentiment_N,etf2_optimism_N,etf2_fear_N,etf2_joy_N,etf2_trust_N,etf2_violence_N,etf2_conflict_N,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,12402.3,0.070054,-0.197907,-0.004435,0.006249,0.005604,-0.004082,0.030236,0.027495,...,0.067871,0.005986,0.015555,0.021109,0.028730,0.600580,-0.015058,-0.006636,-0.000649,0.001178,0.000240,0.022133,0.000697,0.006660,0.000096,0.003702,0.001154,0.001082,NaN,0.002020,0.009328,0.003991,-0.001034,0.001034,0.000192
2009-01-02 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,12053.2,0.069011,-0.197251,-0.002945,0.005019,0.006181,-0.003671,0.029411,0.028665,...,0.068598,0.005567,0.015203,0.021244,0.027310,0.611780,-0.014456,-0.005692,-0.000325,0.001423,0.000100,0.021958,0.000924,0.006615,0.000100,0.003445,0.001048,0.000624,NaN,0.001947,0.009087,0.003994,-0.000549,0.000774,0.000200
2009-01-02 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,11731.3,0.066934,-0.197975,-0.002302,0.004390,0.006095,-0.003687,0.028641,0.029281,...,0.069051,0.005032,0.014614,0.021188,0.025765,0.618360,-0.013575,-0.005487,-0.000202,0.001593,0.000152,0.022316,0.000834,0.006220,0.000101,0.003388,0.000759,0.000076,NaN,0.001745,0.009001,0.003843,0.000000,0.000834,0.000202
2009-01-02 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,11805.9,0.066915,-0.204220,-0.001567,0.004532,0.005379,-0.003769,0.029223,0.029307,...,0.069033,0.005284,0.015578,0.021333,0.026269,0.613965,-0.014258,-0.005184,-0.000174,0.001736,0.000248,0.022414,0.000819,0.006697,NaN,0.003324,0.000645,-0.000099,NaN,0.002034,0.008980,0.003572,0.000248,0.000918,0.000546
2009-01-02 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,12194.3,0.067676,-0.203005,-0.002132,0.004551,0.005248,-0.003977,0.030260,0.029235,...,0.069013,0.005136,0.015746,0.020762,0.027900,0.610689,-0.014268,-0.005136,-0.000313,0.001688,0.000265,0.022681,0.000796,0.007234,NaN,0.003665,0.000844,0.000265,NaN,0.002170,0.009621,0.003496,0.000506,0.000965,0.000531
2009-01-02 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,12217.9,0.067499,-0.202899,-0.002824,0.004379,0.005402,-0.003990,0.030202,0.029424,...,0.069163,0.005241,0.016034,0.020916,0.027737,0.609533,-0.013945,-0.004882,-0.000215,0.001579,0.000120,0.023056,0.000790,0.007132,NaN,0.003590,0.000885,0.000335,NaN,0.002178,0.009549,0.003709,0.000096,0.001197,0.000526
2009-01-02 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,12309.1,0.067799,-0.202777,-0.001584,0.004549,0.005524,-0.003351,0.029369,0.029572,...,0.070543,0.005707,0.016420,0.020606,0.026360,0.608068,-0.013968,-0.004280,0.000140,0.001497,0.000094,0.023900,0.000772,0.006970,NaN,0.003134,0.000819,0.000468,NaN,0.002222,0.008584,0.003859,0.000421,0.001169,0.000702
2009-01-02 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2.0,11729.4,0.065501,-0.197282,-0.002686,0.004945,0.005456,-0.003442,0.028944,0.030266,...,0.070313,0.006225,0.016722,0.020776,0.026277,0.604203,-0.013614,-0.003619,0.000290,0.001400,0.000145,0.024323,0.000603,0.007094,NaN,0.002799,0.000627,0.000651,NaN,0.002389,0.008180,0.004078,0.000603,0.001206,0.000724
2009-01

In [51]:
day = '2009-01-02'
dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
dayex = filter_day(raw_df.loc[dayl[0]:dayl[1]])

In [62]:
get_local_dti(raw_df.loc[dayl[0]:dayl[1]].iloc[:, 0:4].index)

DatetimeIndex(['2009-01-01 19:00:00-05:00', '2009-01-01 20:00:00-05:00',
               '2009-01-01 21:00:00-05:00', '2009-01-01 22:00:00-05:00',
               '2009-01-01 23:00:00-05:00', '2009-01-02 00:00:00-05:00',
               '2009-01-02 01:00:00-05:00', '2009-01-02 02:00:00-05:00',
               '2009-01-02 03:00:00-05:00', '2009-01-02 04:00:00-05:00',
               '2009-01-02 05:00:00-05:00', '2009-01-02 06:00:00-05:00',
               '2009-01-02 07:00:00-05:00', '2009-01-02 08:00:00-05:00',
               '2009-01-02 09:00:00-05:00', '2009-01-02 10:00:00-05:00',
               '2009-01-02 11:00:00-05:00', '2009-01-02 12:00:00-05:00',
               '2009-01-02 13:00:00-05:00', '2009-01-02 14:00:00-05:00',
               '2009-01-02 15:00:00-05:00', '2009-01-02 16:00:00-05:00',
               '2009-01-02 17:00:00-05:00', '2009-01-02 18:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', name='id', freq='H')

In [59]:
raw_df.loc[dayl[0]:dayl[1]].iloc[:, 0:4].dropna(how='all')

,pba_gmtOffset,pba_open,pba_high,pba_low
id,,,,
2009-01-02 14:00:00+00:00,-5.0,1212.74,1216.37,1209.11
2009-01-02 15:00:00+00:00,-5.0,1212.28,1235.19,1211.26
2009-01-02 16:00:00+00:00,-5.0,1234.32,1238.97,1231.04
2009-01-02 17:00:00+00:00,-5.0,1237.50,1246.66,1236.43
2009-01-02 18:00:00+00:00,-5.0,1244.19,1252.06,1243.47
2009-01-02 19:00:00+00:00,-5.0,1249.89,1257.75,1247.29
2009-01-02 20:00:00+00:00,-5.0,1255.58,1266.44,1255.58


In [29]:
out.iloc[:, :24].dropna().groupby(pd.Grouper(freq='D')).size().value_counts()

8    3557
0    2466
7    1649
5      51
6      39
9      20
2       7
3       5
1       2
4       2
dtype: int64

In [22]:
out.groupby(pd.Grouper(freq='D'))['pba_close'].size()

id
1996-01-02 00:00:00+00:00    10
1996-01-03 00:00:00+00:00    24
1996-01-04 00:00:00+00:00    24
1996-01-05 00:00:00+00:00    24
1996-01-06 00:00:00+00:00     0
                             ..
2018-01-04 00:00:00+00:00    24
2018-01-05 00:00:00+00:00    24
2018-01-06 00:00:00+00:00     0
2018-01-07 00:00:00+00:00     0
2018-01-08 00:00:00+00:00    24
Freq: D, Name: pba_close, Length: 8043, dtype: int64

In [43]:
days = ['2009-06-01', '2010-04-24', '2011-01-28', '2011-11-08', '2012-10-09', '2012-11-01', '2014-07-03', '2015-09-02', '2016-03-28', '2017-04-21']

In [44]:
for day in days:
    dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
    display(dayl)
    display(out.loc[dayl[0]:dayl[1]].dropna(how='all'))

('2009-06-01 00:00', '2009-06-01 23:00')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-06-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.081359,0.003799,0.017970,0.016372,0.017514,0.570003,-0.025030,0.000594,-0.000950,0.000438,0.000639,0.023442,0.000256,0.088609,0.000018,0.001552,0.001014,0.000447,0.000018,0.002328,0.005734,0.002465,-0.001479,0.001753,0.000073
2009-06-01 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.080528,0.003740,0.016918,0.016697,0.017901,0.569671,-0.023429,0.001020,-0.000689,0.000515,0.000680,0.023167,0.000276,0.084948,0.000037,0.001636,0.001204,0.000956,0.000018,0.001967,0.005275,0.002536,-0.001388,0.002049,0.000074
2009-06-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.080167,0.003563,0.016465,0.016352,0.018208,0.572590,-0.023364,0.000348,-0.000714,0.000723,0.000836,0.022170,0.000253,0.083878,0.000052,0.001586,0.000950,0.001011,NaN,0.001708,0.004356,0.002744,-0.001429,0.001925,0.000070
2009-06-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.080980,0.003548,0.016345,0.014669,0.016892,0.566373,-0.024438,-0.002566,-0.001879,0.001157,0.000876,0.021249,0.000203,0.093685,0.000056,0.001402,0.000589,0.000729,NaN,0.001641,0.004817,0.003134,-0.001543,0.001774,0.000056
2009-06-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.079954,0.003334,0.015567,0.014149,0.016467,0.558873,-0.023901,-0.003762,-0.002523,0.002165,0.001016,0.021191,0.000211,0.096593,0.000115,0.001456,0.000671,0.001003,NaN,0.001527,0.004893,0.003104,-0.001520,0.001705,0.000051
2009-06-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.079811,0.003182,0.015401,0.013977,0.016449,0.555878,-0.023298,-0.004582,-0.002764,0.002588,0.001024,0.021204,0.000273,0.096412,0.000109,0.001430,0.000806,0.000946,NaN,0.001533,0.004849,0.003279,-0.001533,0.001655,0.000085
2009-06-01 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.079290,0.002971,0.015066,0.014032,0.016015,0.554087,-0.022970,-0.005482,-0.002988,0.002710,0.001028,0.021065,0.000312,0.097753,0.000102,0.001437,0.000932,0.001119,NaN,0.001568,0.004908,0.003323,-0.001716,0.001653,0.000080
2009-06-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.078307,0.002795,0.014628,0.014134,0.015741,0.550013,-0.022566,-0.006234,-0.003171,0.003069,0.001134,0.020998,0.000306,0.099234,0.000118,0.001349,0.001215,0.001596,NaN,0.001564,0.005191,0.003305,-0.001849,0.001575,0.000075
2009-06-01 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.078796,0.002711,0.014427,0.014193,0.015515,0.548421,-0.021856,-0.006301,-0.003080,0.003273,0.001119,0.021005,0.000276,0.100007,0.000125,0.001275,0.001160,0.001524,NaN,0.001545,0.005338,0.003522,-0.001831,0.001795,0.000073


('2010-04-24 00:00', '2010-04-24 23:00')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


('2011-01-28 00:00', '2011-01-28 23:00')

...

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052414,0.008089,0.027585,0.021473,0.030094,0.566653,-0.005433,0.003967,0.000765,0.000174,0.000240,0.032642,0.003258,0.004371,0.000700,0.006240,0.000755,0.003399,NaN,0.000558,0.009370,0.003163,0.000818,0.002115,0.003869
2015-09-02 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052444,0.007983,0.027853,0.021635,0.030574,0.565390,-0.005365,0.004008,0.000753,0.000210,0.000278,0.033059,0.003255,0.004299,0.000697,0.006166,0.000750,0.003369,NaN,0.000550,0.009660,0.003160,0.000733,0.002043,0.003916
2015-09-02 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052575,0.007930,0.027704,0.021871,0.031335,0.564648,-0.006284,0.004357,0.000720,0.000252,0.000277,0.032423,0.003155,0.004242,0.000653,0.005795,0.000708,0.003299,NaN,0.000539,0.010219,0.003191,0.000644,0.001932,0.005183
2015-09-02 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052432,0.007985,0.028419,0.022371,0.031018,0.564152,-0.006048,0.004012,0.000690,0.000255,0.000299,0.032531,0.003165,0.004037,0.000614,0.005758,0.000719,0.003375,NaN,0.000554,0.010164,0.003159,0.000474,0.001909,0.005198
2015-09-02 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052366,0.007958,0.028723,0.022548,0.030981,0.563598,-0.005666,0.003953,0.000645,0.000278,0.000338,0.032173,0.003180,0.004001,0.000616,0.005789,0.000722,0.003452,NaN,0.000543,0.010225,0.003149,0.000521,0.001910,0.005349
2015-09-02 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052489,0.008055,0.029267,0.023114,0.030415,0.567890,-0.005535,0.004059,0.000678,0.000181,0.000254,0.031785,0.003194,0.003954,0.000620,0.005860,0.000665,0.003528,NaN,0.000589,0.010126,0.003086,0.000595,0.001839,0.004864
2015-09-02 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052866,0.008268,0.029744,0.023411,0.030347,0.568449,-0.005868,0.004019,0.000677,-0.000019,0.000153,0.031744,0.003098,0.003930,0.000607,0.005761,0.000629,0.003578,NaN,0.000581,0.010117,0.003177,0.000547,0.001703,0.004753
2015-09-02 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052750,0.008612,0.030423,0.023753,0.030748,0.565856,-0.005748,0.004121,0.000693,-0.000089,0.000073,0.031951,0.003095,0.003956,0.000607,0.005691,0.000604,0.003584,NaN,0.000582,0.010188,0.003235,0.000556,0.001789,0.004703
2015-09-02 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052550,0.008792,0.031272,0.023939,0.030803,0.566284,-0.005493,0.004161,0.000707,-0.000044,0.000086,0.032053,0.003115,0.003942,0.000606,0.005706,0.000647,0.003685,NaN,0.000593,0.010194,0.003076,0.000580,0.001773,0.004726


('2016-03-28 00:00', '2016-03-28 23:00')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-03-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.063551,0.005585,0.023432,0.017383,0.099577,0.492230,-0.006126,0.001276,-0.000099,0.000530,0.000265,0.035022,0.000646,0.002469,0.001989,0.003795,0.001458,0.004375,NaN,0.000265,0.003579,0.001641,0.004607,0.001574,0.000298
2016-03-28 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.063576,0.005788,0.023855,0.017285,0.099853,0.492019,-0.006133,0.001292,-0.000064,0.001052,0.000383,0.034768,0.000558,0.002344,0.002009,0.006028,0.001802,0.004768,NaN,0.000255,0.003157,0.001547,0.004210,0.001515,0.000255
2016-03-28 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064223,0.005824,0.024784,0.017805,0.102491,0.487129,-0.006033,0.001757,0.000032,0.001250,0.000396,0.035612,0.000554,0.002358,0.001994,0.005982,0.001788,0.004748,NaN,0.000253,0.003213,0.001567,0.003767,0.001488,0.000222
2016-03-28 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064869,0.005631,0.025874,0.017321,0.102950,0.490920,-0.006596,0.001583,0.000000,0.001096,0.000320,0.035229,0.000533,0.002481,0.001918,0.006119,0.001598,0.004246,NaN,0.000244,0.003059,0.001507,0.003699,0.001370,0.000213
2016-03-28 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064527,0.005392,0.025708,0.017251,0.103155,0.491448,-0.006518,0.001252,0.000015,0.001090,0.000309,0.035272,0.000560,0.002519,0.001856,0.006099,0.001532,0.004287,NaN,0.000236,0.003153,0.001400,0.003668,0.001267,0.000206
2016-03-28 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064316,0.005175,0.025903,0.016833,0.104123,0.490262,-0.006107,0.001180,-0.000028,0.001265,0.000341,0.035897,0.000512,0.002346,0.001791,0.007208,0.001450,0.004237,NaN,0.000227,0.003412,0.001265,0.003824,0.001308,0.000569
2016-03-28 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.065263,0.005212,0.026263,0.016656,0.105378,0.488203,-0.006453,0.001157,0.000054,0.001184,0.000299,0.036659,0.000490,0.002517,0.001715,0.006940,0.001361,0.004259,NaN,0.000218,0.003565,0.001238,0.003266,0.001252,0.000544
2016-03-28 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.065065,0.005251,0.026137,0.016270,0.108543,0.481856,-0.006454,0.001349,0.000093,0.001124,0.000317,0.037050,0.000476,0.002553,0.001667,0.006944,0.001309,0.004100,NaN,0.000225,0.003704,0.001177,0.003254,0.001376,0.000529
2016-03-28 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.065162,0.005021,0.025330,0.016537,0.109577,0.481362,-0.006330,0.001474,0.000050,0.001112,0.000325,0.037708,0.000874,0.002236,0.001599,0.006707,0.001074,0.003760,NaN,0.000212,0.003922,0.001087,0.003472,0.001124,0.000525


('2017-04-21 00:00', '2017-04-21 23:00')

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks,vol_gmtOffset,vol_open,vol_high,vol_low,vol_close,vol_avgPrice,vol_numTrades,...,etf3_stress,etf3_surprise,etf3_timeUrgency,etf3_uncertainty,etf3_violence,etf3_emotionVsFact,etf3_marketRisk,etf3_longShort,etf3_longShortForecast,etf3_priceDirection,etf3_priceForecast,etf3_volatility,etf3_analystRating,etf3_debtDefault,etf3_dividends,etf3_innovation,etf3_earningsForecast,etf3_fundamentalStrength,etf3_laborDispute,etf3_layoffs,etf3_litigation,etf3_managementChange,etf3_managementTrust,etf3_mergers,etf3_cyberCrime
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-21 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050887,0.007044,0.025219,0.021297,0.032218,0.519128,-0.003865,0.003455,0.000546,0.002920,0.000774,0.029542,0.001219,0.002397,0.000288,0.004831,0.002038,0.003593,NaN,0.000243,0.009833,0.003174,-0.000538,0.003189,0.001544
2017-04-21 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050422,0.007144,0.024980,0.021435,0.032414,0.513587,-0.003454,0.003587,0.000580,0.002866,0.000809,0.029902,0.001263,0.002240,0.000347,0.004835,0.002080,0.003656,NaN,0.000248,0.009792,0.002946,0.000168,0.003255,0.001568
2017-04-21 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050092,0.007221,0.024502,0.021704,0.031765,0.512730,-0.002814,0.003675,0.000588,0.002649,0.000832,0.029898,0.001141,0.002153,0.000347,0.004885,0.002084,0.003767,NaN,0.000240,0.009667,0.002839,0.000336,0.003240,0.001508
2017-04-21 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050287,0.007115,0.023856,0.021869,0.031529,0.514510,-0.003064,0.003768,0.000600,0.002537,0.000841,0.029798,0.001123,0.001964,0.000355,0.004662,0.002041,0.003611,NaN,0.000256,0.009450,0.002832,0.000401,0.003221,0.001502
2017-04-21 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050078,0.007439,0.024358,0.021212,0.031293,0.515094,-0.003263,0.003762,0.000604,0.002408,0.000799,0.029699,0.001093,0.001915,0.000356,0.004557,0.001843,0.003689,NaN,0.000256,0.009610,0.002802,0.000386,0.003161,0.001567
2017-04-21 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050273,0.007590,0.024583,0.019663,0.031660,0.514689,-0.003282,0.003501,0.000551,0.002364,0.000782,0.029707,0.001084,0.001941,0.000366,0.004562,0.001828,0.003829,NaN,0.000238,0.009909,0.002749,-0.000850,0.003047,0.001548
2017-04-21 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.049652,0.007534,0.025821,0.019293,0.031536,0.516379,-0.003568,0.003129,0.000436,0.002049,0.000766,0.029340,0.001013,0.002101,0.000292,0.004431,0.001775,0.003543,NaN,0.000233,0.009513,0.003887,-0.002201,0.002992,0.001524
2017-04-21 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.049730,0.007414,0.025475,0.019256,0.031517,0.517433,-0.003582,0.003013,0.000297,0.002167,0.000776,0.029135,0.001002,0.002197,0.000308,0.004342,0.001826,0.003536,NaN,0.000234,0.009433,0.003900,-0.002249,0.003043,0.001521
2017-04-21 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050968,0.007404,0.024605,0.019137,0.031776,0.519805,-0.003994,0.002817,0.000218,0.002129,0.000766,0.029025,0.001007,0.002207,0.000315,0.004209,0.001714,0.003110,NaN,0.000248,0.009663,0.003891,-0.002940,0.003043,0.001533


In [16]:
ex = raw_df['1996-01-06']
local_times = ex.index + pd.to_timedelta(ex.loc[:, 'pba_gmtOffset'], unit='H')
min_time = local_times.min()

In [19]:
all(pd.isnull(local_times))

True

In [229]:
pd.isnull(min_time)

False

In [135]:
ex = raw_df['1996-01-02']

In [6]:
day = '2010-04-24'
dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
ass = 'dow_jones'
raw_pba = pba[ass]
raw_vol = vol[ass]

In [67]:
raw_pba.loc[dayl[0]: dayl[1]].dropna()

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades,pba_openBid,pba_highBid,pba_lowBid,pba_closeBid,pba_numBids,pba_openAsk,pba_highAsk,pba_lowAsk,pba_closeAsk,pba_numAsks
id,,,,,,,,,,,,,,,,,,
2010-04-24 18:00,-4,11145.33,11146.01,11144.87,11144.95,63066612.0,11145.4560,10.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
2010-04-24 19:00,-4,11145.03,11146.84,11141.93,11145.10,63793084.0,11143.9832,53.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0


In [10]:
DataAPI.axe_load(['hdxret', 'hdxret'])[1][['dow_jones', 'hdxret', 'hdxret', 'pba_hoc_hdxret', 'pba_hoc_hdxret']]['2014-04-23']

,pba_oc
id,
2014-04-23 00:00:00+00:00,NaN
2014-04-23 01:00:00+00:00,NaN
2014-04-23 02:00:00+00:00,NaN
2014-04-23 03:00:00+00:00,NaN
2014-04-23 04:00:00+00:00,NaN
2014-04-23 05:00:00+00:00,NaN
2014-04-23 06:00:00+00:00,NaN
2014-04-23 07:00:00+00:00,NaN
2014-04-23 08:00:00+00:00,NaN


In [17]:
DataAPI.axe_load(['dret', 'dret'])[1][['dow_jones', 'dret', 'dret', 'pba_hoc_hdxret_dret', 'pba_hoc_hdxret_dret']].loc['2014-04-23']

pba_oc   -0.00077
Name: 2014-04-23 00:00:00+00:00, dtype: float64

In [48]:
for ass in ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']:
    hdxret = DataAPI.axe_load(['hdxret', 'hdxret'])[1][[ass, 'hdxret', 'hdxret', 'pba_hoc_hdxret', 'pba_hoc_hdxret']]
    dret = DataAPI.axe_load(['dret', 'dret'])[1][[ass, 'dret', 'dret', 'pba_hoc_hdxret_dret', 'pba_hoc_hdxret_dret']]
    ddir = DataAPI.axe_load(['ddir', 'ddir'])[1][[ass, 'ddir', 'ddir', 'pba_hoc_hdxret_ddir', 'pba_hoc_hdxret_ddir']]
    display(ass, dret[dret['pba_oc']==0])

'sp_500'

,pba_oc
id,
2001-09-11 00:00:00+00:00,0.0


'russell_2000'

,pba_oc
id,
2001-09-11 00:00:00+00:00,0.0


'nasdaq_100'

,pba_oc
id,


'dow_jones'

,pba_oc
id,
1998-04-02 00:00:00+00:00,0.0
